# <center>Antibody – SARS-CoV-2 Molecular Dynamics Setup and energy estimation tutorial</center>

This tutorial aims to illustrate the process of setting up a simulation system containing an antibody and a SARS-CoV-2 protein, step by step, using the SWISS-MODEL, VMD and NAMD. The energy estimation is implemented using AutoDock Tools, the script with commands is provided.

The particular example used is **SARS-CoV-2 protein** (PDB code 6vyb) and **antibody derived from COVID-19 patient.**

The complete antibody sequence:

Heavy chain
QVQLVQSGAAVEKPGASVKVSCRASGYTFTNYGITWVRLAPGRGLEWLGWVTGNGKTRYAETPQGRLTMTTDRSRSTAYMELRSLRSDDTAVYYCARVAANGGL
LWFGGEGDVYFDHWGQGTLLSVSSASTKGPSVFPLAPSSKSTSGGTAALGCLVKDYFPEPVTVSWNSGALTSGVHTFPAVLQSSGLYSLSSVVTVPSSSLGTQT
YICNVNHKPSNTKVDKKVEPKSCDKTHTCPPCPAPELLGGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDGVEVHNAKTKPREEQYNSTYR
VVSVLTVLHQDWLNGKEYKCKVSNKALPAPIEKTISKAKGQPREPQVYTLPPSRDELTKNQVSLTCLVKGFYPSDIAVEWESNGQPENNYKTTPPVLDSDGSFF
LYSKLTVDKSRWQQGNVFSCSVMHEALHNHYTQKSLSLSPGK

Light chain
QSVLTQPPSASGAPGQRITISCSGSASNIGSNYVYWYQVLPGTAPKLLIYSNNQRPSGVPDRFSGSKSGTSASLAISGLRSEDEGDYHCAAWDDSLTGHVAFGG
GTKVTVLGQPKANPTVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADGSPVKAGVETTKPSKQSNNKYAASSYLSLTPEQWKSHRSYSCQVTHEGSTVE
KTVAPTECS
# Settings

## Software used

[SWISS-MODEL:](https://swissmodel.expasy.org/) a fully automated protein structure homology-modelling server

[VMD:](https://www.ks.uiuc.edu/Research/vmd/) a molecular visualization program for displaying, animating, and analyzing large biomolecular systems using 3-D graphics and built-in scripting

[NAMD:](https://www.ks.uiuc.edu/Research/namd/) a parallel molecular dynamics code designed for high-performance simulation of large biomolecular systems

[Chimera:](https://www.cgl.ucsf.edu/chimera/) a program for the interactive visualization and analysis of molecular structures and related data

[AutoDock Tools:](http://autodock.scripps.edu/) is a suite of automated docking tools. Allows estimation of binding energy.
# Pipeline steps

1. Building an antibody from a sequence

2. Superposition of the structure of built antibody and SARS-CoV-2 protein with antibody-SARS-CoV structure (PDB code 2dd8)

3. Minimization of the energy of antibody-SARS-CoV-2 structure

4. Creating a solvent box, solvation and ionization in VMD

5. Minimization of system and heating 

6. Equilibration and MD production run

7. Energy estimation of antibody-SARS-CoV-2 interactions for the last 1 ns of MD 

1. Building an antibody from a sequence

Let's build antibody. We will do it separetely for heavy and light chain. Paste a sequence for heavy chain into "Target sequence" field on [SWISS-MODEL](https://swissmodel.expasy.org/interactive) webpage and click "BUILD MODEL". Repeat the same for the light chain. When the modelling will be completed you can download built models for heavy and light chains. 
<h1><center> SWISS-MODEL interactive workspace</center></h1>
    
<img src="Swiss-Model.png" width="900" height="450">


2. Superposition of the structure of built antibody and SARS-CoV-2 protein with antibody-SARS-CoV structure (PDB code 2dd8)

Now we need to build an antibody-SARS-CoV-2 structure. Let's model antibody-SARS-CoV-2 interactions similar to the interaction mode of SARS-CoV with neutralizing antibody IGG (PDB code 2dd8). We are interested in interactions of antibody with RBD domain of SARS-CoV-2, so let's use only RBD domain of SARS-CoV-2. First, we do superposition of RBD domain of SARS-CoV-2 with antibody-SARS-CoV (PDB code 2dd8), remove SARS-CoV protein and save a structure as Ant-RBD-SARS-CoV-2.pdb<br>
The files containing previously built antibody (Antibody_heavy_chain.pdb, Antibody_light_chain.pdb) and antibody-RBD domain of SARS-CoV-2 complex (Ant-RBD-SARS-CoV-2.pdb) are attached.<br>
Let's do superposition of already build antibody with IGG antibody-RBD domain of SARS-CoV-2 complex in Chimera.<br>
Open Chimera GUI and load Antibody_heavy_chain.pdb and Ant-RBD-SARS-CoV-2.pdb.<br>
Go to Tools -> Structure Comparison -> Match Maker and do superposition of chain H (heavy chain) of Ant-RBD-SARS-CoV-2.pdb with chain H of antibody.
<img src="Chimera_Superposition.png" width="800" height="450">

Repeat the same for a light chain of antibody. Load Antibody_light_chain.pdb and do superposition of chain H (heavy chain) of Ant-RBD-SARS-CoV-2.pdb with chain L of antibody. Remove chains H, L from Ant-RBD-SARS-CoV-2.pdb and save the current structure as Complex.pdb, which will be our working structure of antibody-SARS-CoV-2 complex.

3. Minimization of the energy of antibody-SARS-CoV-2 structure

Let's generate psf file for our built antibody-SARS-CoV-2 complex using VMD and minimize the energy of the structure with NAMD.<br>
Open VMD GUI and load Complex.pdb.<br>
Click Extensions -> Modelling -> Automatic PSF Builder.<br>
It will create Complex_autopsf.psf and Complex_autopsf.pdb files. Copy these files to your working directory.<br>
Now we can start minimization, the minimization.conf file is provided, please check there the path to 'toppar' folder with topology files. 
```bash
> cd your_working_dir
> mkdir min
> path_to_NAMD/namd2 +p8 minimization.conf > minimization.log
```
NAMD will output 'Antibody_min.coor' file with energetically minimized structure in 'min' folder.

4. Creating a solvent box, solvation and ionization in VMD

We will solvate a structure in a rectangular box such that the minimum distance to the edge of the box will be 10 Å under periodic boundary conditions in VMD. Na and Cl ions will be added to neutralize the protein charge, then further ions will be added to mimic a salt solution concentration of 0.15 M.<br> 
Let's load psf and Antibody_min.coor files into VMD. After that click Extensions -> TK Console and type:
```bash
> cd your_working_dir
> package require solvate
> solvate Complex_autopsf.psf min/Antibody_min.coor -t 10 -o solvated
> package require autoionize
> autoionize -psf solvated.psf -pdb solvated.pdb -sc 0.15 -cation SOD -anion CLA -o solvated_ions
```

5. Minimization of system and heating
Let's minimize our system. Type in the Terminal:
```bash
> path_to_NAMD/namd2 +p8 minimization1.conf > minimization1.log
```
In VMD TK Console:
```bash
> cd your_working_dir
> mol load psf solvated_ions.psf namdbin min/Antibody_min1.coor 
> set sel [atomselect top all]
> $sel writepdb solvated_ions_min.pdb
```

Also we need to determine the water box size and estimate coordinate of the center of water box in VMD:

```bash
> set coords [atomselect top all]
> measure minmax $coords
> measure center $coords
```

We need to estimate the box size as max coor - min coor and paste these numbers into files heating.conf, equilibration.conf and md.conf in the field 'Periodic Boundary Conditions'. Paste coordinate of the center of the box into 'cellOrigin' field.<br>
Let's heat our system. In Terminal:
```bash
> mkdir heating
> path_to_NAMD/namd2 +p8 heating.conf > heating.log
```

6. Equilibration and MD production run

Let's equilibrate our systems for 500000 steps (1ns) and run 10 ns of MD.
```bash
> mkdir equil
> path_to_NAMD/namd2 +p8 heating.conf > heating.log
> mkdir md
> path_to_NAMD/namd2 +p8 md.conf > md.log
```

7. Energy estimation of antibody-SARS-CoV-2 interactions for the last 1 ns of MD 

We run 10ns of MD, so we will have 100 frames in total. As we saved trajectories every 50000 steps, for the last 1ns of MD we will have 10 frames. Let's save coordinates of antibody-SARS-CoV-2 structure for the last 10 frames of MD.
Let's load our psf and trajectory file into VMD and save 10 structures that corresponds to the last 10 frames.<br>
In VMD TK Console:
```bash
cd your_working_dir
mol new solvated_ions.psf 
animate read dcd md.dcd
mkdir frames 
for {set i 90} {$i < 100} {incr i} {
  set sel [atomselect top "not water and not ions" frame $i] 
  $sel writepdb frames/frame_$i.pdb
  $sel delete
} 
```

We saved 10 structures, frame_90.pdb-frame_99.pdb into folder 'frames'.
Now we need to analyze each complex and calculate antibody-SARS-CoV-2 interaction energy for each structure.<br>
In order to minimize clashes let's do energy minimization of each complex.<br>
First, we need to generate psf files and then run minimization.<br>
In VMD load frame_90.pdb and generate its psf file Extensions -> Modelling -> Automatic PSF Builder.
Repeat the same for all structures, frame_90.pdb-frame_99.pdb. You will get 10 files, frame_90_autopsf.pdb - frame_99_autopsf.pdb.
Now, we can run minimization for all 10 complexes.<br>
In Terminal:
```bash
cd your_working_dir
./Min_all.sh
```
After minimization we need to convert output files into pdb format.<br>
In VMD TK Console:
```bash
cd your_working_dir/frames
for {set i 1} {$i < 11} {incr i} {
cd $i
mol load psf Complex_autopsf.psf namdbin Antibody_min.coor
set sel [atomselect top all]
$sel writepdb Min.pdb
resetpsf
cd ../
}
```

Now we can estimate energy for our 10 structures.<br>
In Terminal:
```bash
cd your_working_dir
./Energy.sh
```
It will output file energy.txt, which contains frame id and energy of antibody-SARS-CoV-2 interactions for heavy and light chains.